# Fraud Detection with Variational Autoencoder

- VAE는 vanilla 오토인코더에 비해 훨씬 나은 성능의 이상탐지(anomaly detection) 성능을 제공합니다. 실험을 통해 우리는 기존의 AE는 너무 큰 bottleneck을 가지면 identity function이 되며 이상탐지 성능이 떨어지는 것에 반해, VAE는 bottleneck의 크기가 커질수록 이상탐지 성능이 오르는 효과를 갖는 것을 확인할 수 있습니다. 따라서 AE 기반의 anomaly detection을 수행할 때, 기존에는 bottleneck의 크기를 hyper-parameter로 튜닝해야 했던 반면에, VAE의 경우에는 튜닝을 할 필요가 거의 없습니다.  

- 신용 카드 데이터에 VAE(Variational Autoencoder)를 적용합니다. 


- 분포를 배우는 것이 좋은 점은,  
    첫째, 데이터 및 노이즈 생성 프로세스를 명시적으로 모델링함으로써 VAE는 더 robust 하게 그 두가지를 분리하는 방법을 배울 수 있습니다.   
    둘째, disentanglement constraint가 적용되면 잠재 공간이 더 해석 가능해집니다.    
    셋째, 잠재 벡터를 샘플링하여 새 샘플을 생성하고 디코더를 통해 파이프할 수 있습니다.   
    

* disentanglement : latent space가 linear한 구조를 가지게 되어서 하나의 latent vector z 를 움직였을 때 정해진 어떠한 하나의 특성이 변경되게 만들고자 하는 것. 예를 들어, latent vector z 의 specific 한 값을 변경했을 때 생성되는 이미지는 하나의 특성들(머리카락 길이, 성별, 사람의 시선)만 영향을 주게 만들었다고 하면, 이 모델의 latent space는 disentanglement 하다고 말 할 수 있다.

<sup>1</sup> [Building Autoencoders in Keras - Keras Blog](https://blog.keras.io/building-autoencoders-in-keras.html)  
<sup>2</sup> [Variational Autoencoders with Tensorflow Probability Layers - Medium](https://medium.com/tensorflow/variational-autoencoders-with-tensorflow-probability-layers-d06c658931b7)   
<sup>3</sup> [Google Colab VAE Interactive Example](https://colab.research.google.com/github/tensorflow/probability/blob/master/tensorflow_probability/examples/jupyter_notebooks/Probabilistic_Layers_VAE.ipynb#scrollTo=9clSiUTiT3G1)  
<sup>4</sup> [An, J., & Cho, S. (2015). Variational autoencoder based anomaly detection using reconstruction probability. Special Lecture on IE, 2, 1-18.](https://pdfs.semanticscholar.org/0611/46b1d7938d7a8dae70e3531a00fceb3c78e8.pdf)


### Contents 

1. Variational Autoencoder 훈련
2. 잠재 표현 시각화 (Visualize Latent Representations)
3. VAE model 훈련
4. 재구성 오류 계산 
5. 테스트 세트에 대한 평가

## Raw Dataset

model train 은 train set 만으로 합니다. test set은 최종 평가 때까지 사용되지 않습니다.

-  데이터 세트가 매우 불균형합니다.

- Amount 필드는 로그 스케일로 변환합니다.

- class 1 이 너무 적으므로 통상적인 7:3 대신 5:5 비율로 train, test set 분리

## Train a Variational Autoencoder

- latent variable의 prior는 latent dimension의 random unit multivariate normal vector(tfd.MultivariateNormalDiag) 로 설정 합니다.  

- 인코더의 출력인 잠재 분포 parameter 는 0이 아닌 공분산을 가진 다변수 정규정규 분포로 선택 되었습니다. 이는 사기 거래와 정상적인 거래를 분리하는 데 영향을 미치고, 사기 거래의 공분산이 패턴을 가질 수 있기 때문입니다. 따라서, LATENT Dimension 을 2 로 정할 경우, 학습할 distribution parameter는 5개 입니다 (2 개의 평균값 + 하삼각형의 2X2 공분산 행렬에서 가져온 공분산 값 3 개).  


- 디코더의 출력인 데이터 분포 매개 변수는 feature-independent한 정규 분포를 따릅니다.  신용카드 데이터는 실수 값이며 일반적으로 정규 분포를 따르므로 정규 분포를 사용하여 출력을 모델링하는 것이 합리적입니다. 

train set 을 정상과 비정상 set 으로 구분

test set을 정상과 비정상 set으로 구분

### model 정의

fraud case (249개)와 동일한 갯수의 sample을 정상거래와 사기거래에서 sampling 하여 재구성 분포를 시각화 합니다.

### MSE 분포 시각화 비교

## 잠재 표현 (Latent Representations) 시각화

원점 [0,0] 평균 벡터 주변에서 사기와 사기가 아닌 트랜잭션 사이에는 분명한 구분이 있습니다. 이는 VAE가 의미있는 것을 학습하고 있다는 신호입니다.

빨간색으로 표시된 사기 거래의 표준 편차가 훨씬 더 흩어져 있고 사기 거래가 불규칙한 경향이 있다는 직관과 일치합니다.

샘플된 잠재 벡터 z입니다. 클러스터링 및 scattering은 이전 관찰과 일치합니다.

##  Reconstruction error threshold
- 정상 data 로만 train 한 vae model 의 재구성 결과의 원본과의 mse 차이를 기준으로 사기 거래 분류

위에서 보듯 사기성이 아닌 거래의  오류는 사기성 거래 보다 낮습니다.

- mean + 3*std를 threshold로 사용하여 test set의 재구성 error(squared error)가 threshold 보다 크면 fraud 거래로 분류합니다.  
- 1 std - 68%, 2 std - 96%, 3 std - 99.8%

- Test set을 이용하여 재구성 오류 계산  
- thrshold 별로 confusion matrix